In [4]:
from pbp.model import LlamaForCausalLM
from pbp.config import PBPConfig, parse_yaml_to_config

config = parse_yaml_to_config("config.yaml")

model = LlamaForCausalLM.from_pretrained("pickem/best_model", config=config)
model = model.to("cuda")

/root/pickem/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [5]:
import torch
import pickle

# load infer.pkl
with open("infer.pkl", "rb") as f:
    infer = pickle.load(f)

In [6]:
input_ids = torch.tensor(infer["input_ids"]).to("cuda")
teams = (torch.tensor(infer["home_embedding"]).to("cuda"), torch.tensor(infer["away_embedding"]).to("cuda"))

/tmp/ipykernel_44697/3013937646.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  teams = (torch.tensor(infer["home_embedding"]).to("cuda"), torch.tensor(infer["away_embedding"]).to("cuda"))


In [8]:
# add batch_size to input_ids
input_ids = input_ids.unsqueeze(0)


In [9]:
output = model.generate(input_ids=input_ids, team_embeddings=teams, max_length=100)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [10]:
output

tensor([[  5,   7, 183,  23,   8,  26,  10,  27,   9,  27,   9, 101,  98, 101,
          26, 101,   9, 101, 101, 101, 101, 101, 101, 101,   9,  26, 101, 101,
         101,  10,  27,  27,   9,   9, 101,  99,  26, 101, 101,   9, 101, 101,
         101,  26, 101,  10,  27,   9,   9,  27,   9, 101,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1]], device='cuda:0')

In [11]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("pickem_tokenizer")



In [12]:
tokenizer.decode(output[0], skip_special_tokens=False)

'[GAME_START] [ODDS_START] 30 7 [ODDS_END] : & @ [PLAY_SEP] @ [PLAY_SEP] 00 Q1 00 : 00 [PLAY_SEP] 00 00 00 00 00 00 00 [PLAY_SEP] : 00 00 00 & @ @ [PLAY_SEP] [PLAY_SEP] 00 Q3 : 00 00 [PLAY_SEP] 00 00 00 : 00 & @ [PLAY_SEP] [PLAY_SEP] @ [PLAY_SEP] 00 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'